Importing the necessary libraries

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
import plotly.express as px
from vtk import *
from ipywidgets import interactive, Button, HBox, VBox

Reading Image data and updating the output

In [2]:
reader = vtkXMLImageDataReader()
reader.SetFileName("./mixture.vti")
reader.Update()

Finding necessary values to work with data

In [3]:
# Used GetOutput() function to get the data out from the image 
data = reader.GetOutput()

# Getting number of points in the data
numOfPoints = data.GetNumberOfPoints()

# There are three arrays in the data - vtkValidPointMask, vtkGhostType, ImageFile

# Printing range of vtkValidPointMask
print("Range of vtkValidPointMask :", data.GetPointData().GetArray('vtkValidPointMask').GetRange())

# Printing range of vtkGhostType
print("Range of vtkGhostType :", data.GetPointData().GetArray('vtkGhostType').GetRange())

# Printing range of ImageFile
print("Range of ImageFile :", data.GetPointData().GetArray('ImageFile').GetRange())

Range of vtkValidPointMask : (1.0, 1.0)
Range of vtkGhostType : (0.0, 0.0)
Range of ImageFile : (-0.9935540556907654, 0.43280163407325745)


#### In the above we can see that the range of vtkValidPointMask and vtkGhostType can not be worked with as it has no range a single value for all the points. Therefore we will work with 'ImageFile' array

Working with ImageData

In [4]:
# Getting the array 'ImageFile'
arr = data.GetPointData().GetArray('ImageFile')

# Finding the min and max value inside the array
min_val, max_val = arr.GetRange()

Finding out coordinates of each point and value at that point and storing in it the corresponding list

In [5]:
# Emtpy lists
x = []
y = []
z = []
val = []


# Iterating over all the points
for i in range(numOfPoints):
    
    # GetPoint(id) function returns the coordinate of the point with id = id
    tx, ty, tz = data.GetPoint(i)
    
    # Appending coordinates into the lists
    x.append(tx)
    y.append(ty)
    z.append(tz)
    
    # GetValue(id) function returns the value at the point with id = id
    # Appending the value in the val array
    val.append(arr.GetValue(i))

Creating a 3D isosurface using plotly API

#### Reference : 
#### https://plotly.com/python/3d-isosurface-plots/
#### https://plotly.com/python/reference/isosurface/
#### https://plotly.com/python/3d-axes/

In [6]:
def makeIsosurface():
    # x, y, z takes the coordinate of each point
    # colorscale choses the scale of the color in the range 
    # value takes the list of all the values at all the points
    # showscale = False makes sure that the color bar will not be visible
    # isomin and isomax = 0, to make sure that the initial figure should will the isosurface for value = 0
    # cmin and cmax are there to set the bounds of colorscale

    fig = go.Figure(data=go.Isosurface(
            x = x,
            y = y,
            z = z,
            colorscale='BlueRed',
            value=val,
            showscale=False,
            isomin=0,
            isomax=0,
            cmin=min_val,
            cmax=max_val,
    ))

    # Updating the layout of the isosurface
    # disabling ticks in x, y and z axis by using showticklabels=False
    # disabling autosize by using autosize=False
    # setting width and height of the figure to be 500
    fig.update_layout(scene = dict(
                      xaxis = dict(showticklabels = False),
                      yaxis = dict(showticklabels = False), 
                      zaxis = dict(showticklabels = False)), 
                      autosize=False, 
                      width=500,
                      height=500)
    return fig
fig = makeIsosurface()


Creating a histogram using plotly API 
#### Reference : 
#### https://plotly.com/python/histograms/
#### https://plotly.com/python/reference/histogram/

In [7]:
def makeHist(a, b):
    # Created a histogram using px(plotly.express) and making it a graph_objects using go.Figure() function
    # Setting up number of bins in the histogram as 30 to match the number of bins given in the question
    hist = go.Figure(px.histogram(x=val, nbins=30, range_x = (a, b)))

    # Updating the layout of the histogram
    # disabling autosize by using autosize=False
    # setting width and height of the figure to be 500
    # Changing the title in the x and y axis as given in the question
    hist.update_layout(autosize=False, 
                       width=500, 
                       height=500, 
                       xaxis_title='Vortex scalar values', 
                       yaxis_title='Frequency')
    return hist
hist = makeHist(min_val, max_val)

Creating a floating point widget using ipywidgets

#### Reference : 
#### https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#floatslider
#### https://ipywidgets.readthedocs.io/en/stable/examples/Output%20Widget.html

In [8]:
# Making a floating point slider
# Adding a default value of 0 in the slider
# Setting up the bounds of the slider using min and max
# Stepsize is set to be 0.1 in the slider
# Added description of the slider using description parameter
# Set disabled = False to let it enabled
# Set continuous_update = False so that it is updated only when the user unclick the slider
# Set orientation as the horizontal as per the design given in the question
# Set readout = True to show the current value of the float slider beside it
# Format of readout is set to be '.2f' to show two places after decimal
fl_range = widgets.FloatSlider(
    value=0,
    min=min_val,
    max=max_val,
    step=0.01,
    description='Isoval:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',)

Creating a Button widget using ipywidgets

#### Reference :  https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#button

In [9]:
button = widgets.Button(description="Reset")

Making the figure as the widgets using FigureWidget function

#### Reference : https://plotly.com/python/figurewidget/

In [10]:
wfig = go.FigureWidget(fig)
whist = go.FigureWidget(hist)

global variable to track the isovalue at any point 

In [11]:
isoval = None
reset = True

Default Setting to start with

In [12]:
# For default setting the isoval value will be set to isoVal which is passed as a parameter
# Range of histogram is set as the whole range of values in the ImageFile array
# Number of bins is set as 30
# The isomax and isomin value for the isosurface is set as 0 to start with
def defaultSetting(isoVal):
    isoval = isoVal
    fl_range.value = isoval
    hist.update_xaxes(range=[min_val, max_val])
    hist.update_traces(nbinsx=30)
    fig.update_traces(overwrite=True, isomax=0, selector=dict(type='isosurface'))
    fig.update_traces(overwrite=True, isomin=0, selector=dict(type='isosurface'))


Utility Functions

In [13]:
# Function to change the histogram whenever there is a change in the value of the slider
def changeHist(isoval):
    global reset
    if(reset == False):
        # Find the data values which belong in the interval [isoval-0.25, v <= isoval+0.25]
        tempv = []
        for i in range(len(val)):
            if(val[i] >= (isoval-0.25) and val[i] <= (isoval+0.25)):
                tempv.append(val[i])

        # Updating the histogram
        hist.update_traces(x = tempv, nbinsx=30)
        hist.update_xaxes(range=[isoval-0.25, isoval+0.25])
    else:
        hist.update_traces(x = val, nbinsx=30)
        hist.update_xaxes(range=[min_val, max_val])
        
    reset = False
    # returning the updated histogram
    return go.FigureWidget(hist)

# Function to change the isosurface whenever there is a change in the value of the slider
def changeIsoSurface(isoval):
    # Updating the isomax value
    fig.update_traces(isomax=isoval, selector=dict(type='isosurface'))
    
    # Updating the isomin value
    fig.update_traces(isomin=isoval, selector=dict(type='isosurface'))
    
    # return the update figure
    return go.FigureWidget(fig)

# Function to display the slider and reset button horizontally
def displaySliderButton():
    return HBox([fl_range, button])

# Function to display the left part of the target view
def displayLeftPart(wfig):
    return VBox([displaySliderButton(), wfig])

# Function to display both left and right part of the view
def displayWidgets(whist, wfig):
    global reset
    reset = False
    display(HBox([displayLeftPart(wfig), whist]))

Callback function of slider

#### Reference : https://ipython.org/ipython-doc/3/api/generated/IPython.display.html#functions

In [14]:
# Callback function whenever there is a change in the value of slider
def change(x):

    # updating the isoval value by the new value of the slider
    isoval = x.new

    # change the histogram
    whist = changeHist(isoval)
    
    # change the isosurface
    wfig = changeIsoSurface(isoval)
    
    # clear the output
    # wait = True makes sure that when the next output is ready till then do not clear the present output
    clear_output(wait = True)
    
    # display the whole view
    displayWidgets(whist, wfig)

Callback function of button

In [15]:
# Callback function whenever the button is clicked
def on_button_clicked(b):
    global reset
    reset = True
    fl_range.value = 0.0
    

Setting up call back functions
#### Reference : https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Events.html

In [16]:
# Setting up callback function for the change in value of the slider
fl_range.observe(change, names='value')

# Setting up callback function for the click in the reset button
button.on_click(on_button_clicked)

Making sure that the settings applied before running the code and calling the display function

In [20]:
defaultSetting(0.0)
clear_output()
displayWidgets(whist, wfig)